In [ ]:
#Import ultralytics
!pip install ultralytics
import ultralytics
ultralytics.checks()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.0/566.0 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.1/203.1 kB 4.5 MB/s eta 0:00:00


In [ ]:
# Import required libraries
import os
import shutil
import random

#!pip install tqdm --upgrade
from tqdm.notebook import tqdm

In [ ]:
# Connect to google drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
#Assign model and images to predict
from ultralytics import YOLO
model=YOLO('/content/drive/MyDrive/UU_Thesis/yolov8/training_results/goats13_6_paint_grande24/weights/best.pt')
source='/content/drive/MyDrive/UU_Thesis/yolov8/selected_images'
#source='/content/drive/MyDrive/UU_Thesis/yolov8/_Test_data_/Total_images'
print(type(model))

<class 'ultralytics.yolo.engine.model.YOLO'>


In [ ]:
#Predict images
results = model(source)



    WARNING ⚠️ stream/video/webcam/dir predict source will accumulate results in RAM unless `stream=True` is passed,
    causing potential out-of-memory errors for large sources or long-running streams/videos.

    Usage:
        results = model(source=..., stream=True)  # generator of Results objects
        for r in results:
            boxes = r.boxes  # Boxes object for bbox outputs
            masks = r.masks  # Masks object for segment masks outputs
            probs = r.probs  # Class probabilities for classification outputs

image 1/1456 /content/drive/MyDrive/UU_Thesis/yolov8/_Test_data_/Total_images/1.jpg: 640x640 1 Goat, 652.5ms
image 2/1456 /content/drive/MyDrive/UU_Thesis/yolov8/_Test_data_/Total_images/10.jpg: 640x640 (no detections), 525.4ms
image 3/1456 /content/drive/MyDrive/UU_Thesis/yolov8/_Test_data_/Total_images/100.jpg: 640x640 (no detections), 540.3ms
image 4/1456 /content/drive/MyDrive/UU_Thesis/yolov8/_Test_data_/Total_images/1000.jpg: 640x480 2 Goats, 406.4m

# Storing results

In [ ]:
# Set the directory where the file will be saved
directory = "/content/drive/MyDrive/UU_Thesis/yolov8/output/labeled_bboxes_totalImg"

os.makedirs(directory)

for n, result in enumerate(results):
    boxes = str(result.boxes.xywh)     #xyxy format /or xywh format
    #print("For result ", n, result,"data =",boxes)
    #print("Boxes Conf = ", result.boxes.conf)

    #Get image path
    path = result.path
    filename = path[path.rfind("/") + 1:]

    new_extension = '.txt'

    if '.' in filename:
        # Split the filename into the base name and extension
        parts = filename.rsplit('.', 1)

        # Replace the extension with the new extension
        new_filename = f"{parts[0]}{new_extension}"
    else:
        # If the filename doesn't have an extension, simply append the new extension to the end
        new_filename = f"{filename}{new_extension}"

    #print(new_filename)

    # Construct the full path to the new file
    new_file_path = os.path.join(directory, new_filename)

    # Open the new file for writing
    with open(new_file_path, "w") as f:
        # Write the contents of 'boxes' to the file
        f.write(boxes)

## Segment Anything

In [ ]:
import os
HOME = os.getcwd()
print("HOME:", HOME)

%cd {HOME}

# Install Segment Anything
import sys
!{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'

# Install jupyter widget
!pip install -q jupyter_bbox_widget roboflow dataclasses-json supervision

# Retrieve weights from Meta
%cd {HOME}
!mkdir {HOME}/weights
%cd {HOME}/weights

!wget -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

CHECKPOINT_PATH = os.path.join(HOME, "weights", "sam_vit_h_4b8939.pth")
print(CHECKPOINT_PATH, "; exist:", os.path.isfile(CHECKPOINT_PATH))

# Create data folder
%cd {HOME}
!mkdir {HOME}/data
%cd {HOME}/data

HOME: /content
/content
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-t4_g2ykt
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-t4_g2ykt
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e27956588
  Preparing metadata (setup.py) ... done
  Created wheel for segment-anything: filename=segment_anything-1.0-py3-none-any.whl size=36589 sha256=469dece007abab0ae7cbf058b701434091299d0360ce43d2d7aa95d6ca326378
  Stored in directory: /tmp/pip-ephem-wheel-cache-41qcku5e/wheels/10/cf/59/9ccb2f0a1bcc81d4fbd0e501680b5d088d690c6cfbc02dc99d
Successfully built segment-anything
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.8/367.8 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 kB 8.3 MB/s 

Load model

In [ ]:
import torch
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
MODEL_TYPE = "vit_h"

sam = sam_model_registry[MODEL_TYPE](checkpoint=CHECKPOINT_PATH).to(device=DEVICE)

##Loading mask predictor

In [ ]:
mask_predictor = SamPredictor(sam)

##Loading image

In [ ]:
def yolo_convertor(yolo_tensor):
  boxes = []
  for box in yolo_tensor:
      x_center, y_center, width, height = box
      x1 = x_center - width / 2
      y1 = y_center - height / 2
      x2 = x_center + width / 2
      y2 = y_center + height / 2
      boxes.append([x1, y1, x2, y2])

  boxes = np.array(boxes)
  return boxes

In [ ]:
import os
import cv2
import numpy as np
import re

# Connect to google drive
from google.colab import drive
drive.mount('/content/drive')

path_to_selected_images = "/content/drive/MyDrive/UU_Thesis/yolov8/_Test_data_/selected_images"
path_to_bounding_boxes = "/content/drive/MyDrive/UU_Thesis/yolov8/output/output_selected_grande"

def load_images_and_tensors():
    image_files = os.listdir(path_to_selected_images)
    text_files = os.listdir(path_to_bounding_boxes)

    # Select only text files that have a corresponding image file name without checking the extension
    text_files = [text_file for text_file in text_files if os.path.splitext(text_file)[0] in image_files]

    data = []

    for image_file in image_files:
        # Load image
        image_path = os.path.join(path_to_selected_images, image_file)
        image = cv2.imread(image_path)

        # Load corresponding text file
        text_file = os.path.splitext(image_file)[0] + '.txt'
        text_path = os.path.join(path_to_bounding_boxes, text_file)
        print("text_path:", text_path)

        with open(text_path, 'r') as f:
            tensor_str = f.read()
            # Remove non-numeric characters and extra spaces
            tensor_str = re.sub(r'[^0-9.,\s]', '', tensor_str).strip()
            # Convert string to numpy array
            try:
              tensor = np.array([list(map(float, line.split(','))) for line in tensor_str.split('\n') if line.strip()])
            except:
              continue


        data.append({'image': image, 'tensor': tensor, 'image_path': image_path})

    return data

Mounted at /content/drive


In [ ]:
yolo_test_data = load_images_and_tensors()
yolo_convertor(yolo_test_data[2]['tensor'])[0]

text_path: /content/drive/MyDrive/UU_Thesis/yolov8/output/output_selected_grande/97.txt
text_path: /content/drive/MyDrive/UU_Thesis/yolov8/output/output_selected_grande/1250.txt
text_path: /content/drive/MyDrive/UU_Thesis/yolov8/output/output_selected_grande/1455.txt
text_path: /content/drive/MyDrive/UU_Thesis/yolov8/output/output_selected_grande/820.txt
text_path: /content/drive/MyDrive/UU_Thesis/yolov8/output/output_selected_grande/1010.txt
text_path: /content/drive/MyDrive/UU_Thesis/yolov8/output/output_selected_grande/931.txt
text_path: /content/drive/MyDrive/UU_Thesis/yolov8/output/output_selected_grande/26.txt
text_path: /content/drive/MyDrive/UU_Thesis/yolov8/output/output_selected_grande/18.txt
text_path: /content/drive/MyDrive/UU_Thesis/yolov8/output/output_selected_grande/171.txt
text_path: /content/drive/MyDrive/UU_Thesis/yolov8/output/output_selected_grande/1049.txt
text_path: /content/drive/MyDrive/UU_Thesis/yolov8/output/output_selected_grande/1093.txt
text_path: /content

array([175.5481 , 225.00165, 418.7071 , 416.78835])

In [ ]:
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

def seperate(detections, image_bgr):

  # Get the mask
  mask = detections.mask
  # Squeeze any singleton dimensions (if there's only one mask)
  mask = np.squeeze(mask)

  # Ensure the mask is a boolean mask
  mask = mask.astype(bool)

  # Create a new image with 4 channels (RGB + alpha)
  cutout = np.zeros((image_bgr.shape[0], image_bgr.shape[1], 4), dtype=np.uint8)

  # Set RGB values
  cutout[..., :3] = image_bgr

  # Set alpha channel
  cutout[..., 3] = mask.astype(np.uint8) * 255  # Set alpha to 255 for non-transparent regions

  # Convert the image from BGR to RGBA
  cutout_rgba = cv2.cvtColor(cutout, cv2.COLOR_BGRA2RGBA)

  # Save the image with transparency to a file (e.g., PNG)
  output_file = 'output.png'
  Image.fromarray(cutout_rgba).save(output_file)

  # # Display the image using matplotlib
  img = Image.open(output_file)
  # plt.imshow(img)
  # plt.axis('off')  # Optional: Turn off axes
  # plt.show()

  # Save image to google drive
  cv2.imwrite(os.path.join("/content/drive/MyDrive/UU_Thesis/segmented_images/CutOut", os.path.basename(image['image_path'])), img) #segmented_image

In [ ]:
#1
import numpy as np
import cv2
import supervision as sv
from jupyter_bbox_widget import BBoxWidget
widget = BBoxWidget()

# default_box is going to be used if you will not draw any box on image above
default_box = {'x': 68, 'y': 247, 'width': 555, 'height': 678, 'label': ''}

for image in yolo_test_data:
    box = widget.bboxes[0] if widget.bboxes else default_box
    box = yolo_convertor(image['tensor'])[0]

    image_bgr = cv2.imread(image['image_path'])
    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

    mask_predictor.set_image(image_rgb)

    masks, scores, logits = mask_predictor.predict(
        box=box,
        multimask_output=True
    )


    #box_annotator = sv.BoxAnnotator() #color=sv.Color.red()
    mask_annotator = sv.MaskAnnotator(color=sv.Color(0,0,0)) #color=sv.Color.red()

    detections = sv.Detections(
        xyxy=sv.mask_to_xyxy(masks=masks),
        mask=masks
    )
    #print(detections)
    #with open('readmeDetection123.txt', 'w') as f:
    #    f.writelines(detections)
    detections2 = detections[detections.area == np.max(detections.area)]

    #source_image = box_annotator.annotate(scene=image_bgr.copy(), detections=detections2, skip_label=True)
    segmented_image = mask_annotator.annotate(scene=image_bgr.copy(), detections=detections2)

    seperate(detections, image_bgr)

    # Save image to google drive
    cv2.imwrite(os.path.join("/content/drive/MyDrive/UU_Thesis/segmented_images", os.path.basename(image['image_path'])), segmented_image) #segmented_image


ValueError: ignored

In [ ]:
detections2

# with open('readmeDet.txt', 'w') as f:
#     f.write(string)

#print(string)


NameError: ignored